# Chroma

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader('text.txt')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=50,
    encoding_name='cl100k_base'
)

texts = text_splitter.split_text(data[0].page_content)
texts[0]

'한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.\n\n고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.'

In [3]:
embeddings_model = OpenAIEmbeddings()
db = Chroma.from_texts(
    texts, 
    embeddings_model,
    collection_name = 'history',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

db

거리 지정 방식을 생성시에 딕셔너리 인자로 넣어줌

In [4]:
query = '누가 한글을 창제했나요?'
docs = db.similarity_search(query)
print(docs[0].page_content)

조선은 1392년 이성계에 의해 건국되어, 1910년까지 이어졌습니다. 조선 초기에는 세종대왕이 한글을 창제하여 백성들의 문해율을 높이는 등 문화적, 과학적 성취가 이루어졌습니다. 그러나 조선 후기에는 내부적으로 실학의 발전과 함께 사회적 변화가 모색되었으나, 외부로부터의 압력은 점차 커져만 갔습니다.


## MMR

In [24]:
# Load data -> Text split

from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = PDFPlumberLoader('/home/yhkim/code/Langchain_practice/000660_SK_2023.pdf')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base',
)

documents = text_splitter.split_documents(data)
len(documents)

37

In [26]:
# Embedding -> Upload to Vectorstore
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
db2 = Chroma.from_documents(
    documents, 
    embeddings_model,
    collection_name = 'esg',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

db2

In [27]:
query = 'ESG 전략을 알려줘'
docs = db2.similarity_search(query)
print(len(docs))
print(docs[0].page_content)

4
· 실 사 대상 기업 산업 및 규모에 따른 · 전 체 포트폴리오를 16개 업종, 기업 규모에 따라 3개 그룹으로 구분 · 탄 소 규제/가격 변동에 따른 재무영향 점검 · 인 수 이후 ESG Value-up 기반 Exit 전략 도출
체크리스트 생성 ESG 중점관리 항목 도출 [수익성] 매출액/영업이익 대비 탄소 비용 추이 고려 · 중 대 ESG 리스크/기회 현황 및
· 점 검항목은 유형에 따라 · E SG 외부평가 및 주가 상관관계 상위 영역 분석에 따라 [경제성] 투자 대비 감축 수단 효과 검토 ESG 관리·공시 수준 확인
ESG 리스크와 관리체계로 구분 산업 핵심관리 영역 도출 [시장성] 경쟁사 대비 속도/수준 감안 대응전략 점검
· 서 면진단 및 현장실사 실시 정기 평가 실시 물리적 리스크
셀사이드(Sell-side) 점검사항 관리
시 사점 도출 · 연 2회 ESG 실적점검 실시, · 자 산 소재 지역의 취약한 이상기후 요인 식별
· V aluation 반영을 통한 중요 ESG 리스크/기회 관련 이슈 식별 · 고 위험 기업 대상 관리방안 마련 권고 중대 ESG 이슈 존재 여부 검토
인수가액 조정 · 매 각 대상 시장 내 ESG 규제 준수 여부 확인
· P MI(Post-Merger Integration, · E SG 우수 영역에 대한 정보공개 및
인수 합병 후 통합 과정) ESG 기반 주주 소통 기회 확대 방안 제시
개선 과제 제시 주 주 소통 대상 안건 · 국 내외 책임투자 기준 부합 여부 확인
· 해 당 산업 ESG 중점관리 항목과 연관된 리스크가 식별되었거나, · 우 수 관리 영역 정보공개 및
연중 중대한 ESG Controversy Issue(예상하지 못한 우려) 발생 시 이해관계자 커뮤니케이션
투자회사 유형별 관여 방식
· 이 사회 기타비상무이사를 통한 소통
01 02 03
· E SG 점검 리포트 또는 주주서한 발송
Pre-Acquisition (인수 전) Value-up Period (보유 기간) Exit (투자 회수)


In [28]:
print(docs[-1].page_content)

12 ESG Special Report 2023 Where we are heading | How we get there | What we are preparing 13
OUR PORTFOLIO SK주식회사는 ESG기반의 4대 핵심사업을 중심으로
글로벌 투자 현황 (조원) 재무 성과 (2022년 별도 기준)
장기적 관점의 투자 전략을 실행하고 있습니다.
AND 2022년에는 앵커 자산과 신규 투자를 연계한 투자
EU 동남아 등
성과로 매출액과 영업이익 성장을 이루어 냈습니다. 0.6(7%) 0.1(2%) YoY +24%
PERFORMANCES 또한 투명한 ESG 정보공개와 이해관계자 소통으로
대외 유수 평가기관으로부터 최고 수준의 3.4
중국 조원
북미
평가 등급을 획득하며 ESG 역량을 인정받았습니다. 0.8(9%)
3.5(41%)
8.6 조원 *
한국
YoY +33%
3.5(41%)
ESG 기반 4대 핵심 영역 포트폴리오
1.1
조원
북미 | Plug Power, TerraPower
첨단소재 그린
한국 | SK스페셜티, SK실트론, SK바이오팜, SK시그넷
고성장 ‵반도체/전력반도체/ GREEN 산업의 성장 거점 선점
중국 | ESR, ChinData
배터리′ 핵심 소재 선제적 투자로 및 넷제로 실현을 위한 탈탄소
EU | Yposkesi
기존 사업과의 시너지 창출 및 기술 기업 자산 확보
동남아 등
고성장 소재 포트폴리오 확대
가속화 투자 자산 * 2 022년까지 누적 투자 금액(성장 영역 Inc. 직접 투자 자산)으로 매출 영업이익
TerraPower, Plug Power, 합병 비용 포함 및 Divest 금액 제외
●
Monolith, Atom Power,
투자 자산
Lunar Energy, Fulcrum,
SK스페셜티, SK실트론,
●
Wason, SES AI, 3.9 1.9 Joyvio, Perfect Day, ESG 성과 (2023년 상반기 기준)
조원 조원 Nature’s Fynd,
SK시그넷, SK파워텍
Wildtype
8.6 조원*
지속가능경

In [29]:
mmr_docs = db2.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

4
· 실 사 대상 기업 산업 및 규모에 따른 · 전 체 포트폴리오를 16개 업종, 기업 규모에 따라 3개 그룹으로 구분 · 탄 소 규제/가격 변동에 따른 재무영향 점검 · 인 수 이후 ESG Value-up 기반 Exit 전략 도출
체크리스트 생성 ESG 중점관리 항목 도출 [수익성] 매출액/영업이익 대비 탄소 비용 추이 고려 · 중 대 ESG 리스크/기회 현황 및
· 점 검항목은 유형에 따라 · E SG 외부평가 및 주가 상관관계 상위 영역 분석에 따라 [경제성] 투자 대비 감축 수단 효과 검토 ESG 관리·공시 수준 확인
ESG 리스크와 관리체계로 구분 산업 핵심관리 영역 도출 [시장성] 경쟁사 대비 속도/수준 감안 대응전략 점검
· 서 면진단 및 현장실사 실시 정기 평가 실시 물리적 리스크
셀사이드(Sell-side) 점검사항 관리
시 사점 도출 · 연 2회 ESG 실적점검 실시, · 자 산 소재 지역의 취약한 이상기후 요인 식별
· V aluation 반영을 통한 중요 ESG 리스크/기회 관련 이슈 식별 · 고 위험 기업 대상 관리방안 마련 권고 중대 ESG 이슈 존재 여부 검토
인수가액 조정 · 매 각 대상 시장 내 ESG 규제 준수 여부 확인
· P MI(Post-Merger Integration, · E SG 우수 영역에 대한 정보공개 및
인수 합병 후 통합 과정) ESG 기반 주주 소통 기회 확대 방안 제시
개선 과제 제시 주 주 소통 대상 안건 · 국 내외 책임투자 기준 부합 여부 확인
· 해 당 산업 ESG 중점관리 항목과 연관된 리스크가 식별되었거나, · 우 수 관리 영역 정보공개 및
연중 중대한 ESG Controversy Issue(예상하지 못한 우려) 발생 시 이해관계자 커뮤니케이션
투자회사 유형별 관여 방식
· 이 사회 기타비상무이사를 통한 소통
01 02 03
· E SG 점검 리포트 또는 주주서한 발송
Pre-Acquisition (인수 전) Value-up Period (보유 기간) Exit (투자 회수)


In [30]:
print(mmr_docs[-1].page_content)

21
20
ESG Special Report 2023
투
자
 
검
토
 
단
계
Pre-Acquisition (인수 전)
01
포
트
폴
리
오
 
E
S
G
 
관
리
 
체
계
장
기
적
 
관
점
에
서
 
기
업
가
치
 
제
고
를
 
실
현
하
기
 
위
해
 
핵
심
자
산
인
 
투
자
 
포
트
폴
리
오
의
 
경
제
적
 
가
치
와
 
함
께
 
E
S
G
 
가
치
를
 
통
합
적
으
로
 
관
리
하
기
 
위
한
 
체
계
를
 
구
축
하
고
 
있
습
니
다
.
투
자
 
검
토
 
시
점
부
터
 
인
수
 
후
,
 
회
수
 
시
점
까
지
 
투
자
L
i
f
e
 
C
y
c
l
e
에
 
걸
쳐
 
적
용
되
는
 
체
계
적
인
 
E
S
G
 
관
리
를
 
기
반
으
로
 
내
부
적
으
로
는
 
E
S
G
를
 
고
려
한
 
합
리
적
인
 
투
자
의
사
 
결
정
을
 
이
행
하
고
,
 
시
장
에
서
는
 
포
트
폴
리
오
의
 
기
업
가
치
가
 
시
장
에
서
 
제
대
로
 
평
가
받
으
며
 
나
아
가
 
사
회
·
환
경
에
 
미
치
는
 
파
급
력
을
 
높
일
 
수
 
있
도
록
 
노
력
하
겠
습
니
다
.
포
트
폴
리
오
 
E
S
G
 
관
리
 
원
칙
S
K
주
식
회
사
투
자
회
사
기
업
가
치
 
관
점
의
E
S
G
 
중
점
관
리
 
항
목
 
도
출
자
사
 
E
S
G
 
관
리
전
략
E
S
G
 
성
과
 
데
이
터
 
관
리
기
업
가
치
와
E
S
G
 
성
과
 
연
계
성
 
분
석
포
트
폴
리
오
 
E
S
G
 
관
리
전
략
 
U
p
g
r
a
d
e
성
장
단
계
산
업
특
성
E
S
G
 
D
i
v
e
s
t
m
e
n
t
 
전
략
 
검
토
 
 
 
 
 
 
 
E
S
G


## using metadata

In [31]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.",
        metadata={
            "title": "LangChain 소개",
            "author": "AI 개발자",
            "url": "http://example.com/langchain-intro"
        }
    ),
    Document(
        page_content="벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.",
        metadata={
            "title": "벡터 데이터베이스 개요",
            "author": "데이터 과학자",
            "url": "http://example.com/vector-db-overview"
        }
    )
    # 추가 문서들...
]

In [32]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings()

# Chroma 벡터 스토어에 문서와 메타데이터 저장
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # 벡터 스토어를 디스크에 저장
)

In [33]:
query = "LangChain이란 무엇인가요?"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print(f"내용: {doc.page_content}")
    print(f"제목: {doc.metadata['title']}")
    print(f"저자: {doc.metadata['author']}")
    print(f"URL: {doc.metadata['url']}")
    print("---")

내용: LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.
제목: LangChain 소개
저자: AI 개발자
URL: http://example.com/langchain-intro
---
내용: 벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.
제목: 벡터 데이터베이스 개요
저자: 데이터 과학자
URL: http://example.com/vector-db-overview
---


In [34]:
# 벡터스토어 db 인스턴스를 생성
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)


vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )
vectorstore

/tmp/ipykernel_2798/2040709931.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
/home/yhkim/env/RAG/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

FAISS는 거리함수 유사도 검색 설정법만 체크하자